<a href="https://colab.research.google.com/github/zhaooao/ai./blob/main/cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 驗證集
將訓練集部分保留，用於模型驗證

In [1]:
from sklearn.svm import SVC # "Support vector classifier"
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

這裡我們用鸢尾花 (Iris) 多分类数据集測試。

（課外閱讀：https://blog.csdn.net/u013172930/article/details/146293440 ）

In [2]:
iris = load_iris()
X, y = iris.data, iris.target

進行模型驗證的一種方法是使用所謂的保留集 (holdout set)：也就是說，我們保留一些資料子集，然後使用這個保留集來檢查模型效能。

In [3]:
# split the data with 50% in each set
X1, X2, y1, y2 = train_test_split(X, y, random_state=0, test_size=0.5)

然後我們選擇一個模型，在一組資料（訓練集（X1，y1））上擬合該模型，並在第二組資料（驗證集（X2，y2））上評估該模型

In [4]:
model = SVC(kernel='rbf', gamma=0.01, C=10)
# fit the model on one set of data
model.fit(X1, y1)

# evaluate the model on the second set of data
y2_model = model.predict(X2)
accuracy_score(y2, y2_model)

0.9466666666666667

Q: 使用保留集進行模型驗證有何缺點？

A: 使用保留集進行模型驗證的一個缺點是，我們會在模型訓練中失去一部分資料。在上述情況下，一半的資料集對模型訓練沒有貢獻！這並非最佳選擇，並且可能會導致問題——尤其是在初始訓練資料集較小的情況下。

## 交叉驗證 (cross validation)

解決保留方法缺點的一種方法是使用交叉驗證；即進行一系列擬合，其中資料的每個子集既用作訓練集，又用作驗證集。

這裡我們進行了兩次驗證試驗，交替使用每一半資料作為保留集。

In [5]:
# fit the model on one set of data
y2_model = model.fit(X1, y1).predict(X2)
y1_model = model.fit(X2, y2).predict(X1)
accuracy_score(y1, y1_model), accuracy_score(y2, y2_model)

(0.9866666666666667, 0.9466666666666667)

最終結果是兩個準確率分數，我們可以將它們合併（例如取平均值），從而更好地衡量模型的整體表現。這種特殊的交叉驗證形式是雙重交叉驗證，將資料分成兩組，並分別用作驗證集。

我們可以擴展這個想法，使用更多的試驗，以及更多的數據折疊。

這裡我們將資料分成五組，並依序使用每一組來評估其餘 4/5 資料的模型適配度。

In [6]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])